# * Import : "CDSAPPO.DIM_TIME"
    WHERE TM_KEY_YR >= 2023

## Parameter

In [3]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 2, 17)

### Input Parameter

In [4]:
# Input : 0 for dummy parameter

op_dir = 'data'
op_file = 'dim_time.csv'
v_param = dict(yr_start=2023, yr_end=0)

## ETL Process...

### Step 1 : DIM_TIME
    DB -> CSV file

In [5]:
job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-DIM_TIME.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()

# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
print(f'\n{AKPIPRD_db} : Connected')
tgt_cur = tgt_conn.cursor()


try:
    print(f"\nProcessing...\n\n   -> data as of (YR_Start: {v_param['yr_start']}, YR_End: {v_param['yr_end']})")

    # Create Dataframe
    src_cur.execute(query, v_param)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
    src_cur.close()

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_file}" successfully')
    
    # Truncate
    tgt_cur.execute(f"TRUNCATE TABLE AUTOKPI.DIM_TIME")
    print(f'\n   -> TRUNCATE : "DIM_TIME" : Done !')
    
    # Insert
    tgt_cur.executemany(f"""
        INSERT INTO AUTOKPI.DIM_TIME
        (TM_KEY_DAY, DATE_ID, DATE_VALUE, DATE_DESC, DAY_DESC, DAY_SHORT, DAY_DESC_TH, DAY_SHORT_TH, DAY_NO, DAY_OF_WEEK, DAYS_IN_MONTH, TM_KEY_WK, WEEK_DESC, WEEK_YEAR, TRUE_TM_KEY_WK, TRUE_WEEK, TRUE_WEEK_DESC, TM_KEY_MTH, MONTH_ID, MONTH_SHORT, MONTH_DESC, MONTH_SHORT_TH, MONTH_DESC_TH, MONTH_NO, MONTH_BEG_DATE, MONTH_END_DATE, TM_KEY_QTR, QUARTER_DESC, QUARTER_SHORT, QUARTER_DESC_TH, QUARTER_SHORT_TH, QUARTER_NO, TM_KEY_YR, YEAR_DESC, YEAR_SHORT, YEAR_DESC_TH, YEAR_SHORT_TH, PERIODFLAG, ETL_DATE, ETL_UPDATE, WEEK_MONTH, WEEK_MONTH_DESC, TRUE_WK_BEG_DATE, TRUE_WK_END_DATE, TRUE_WK_MTH_BEG_DATE, TRUE_WK_MTH_END_DATE) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42,:43,:44,:45,:46)
    """, rows)
    print(f'\n   -> INSERT : "DIM_TIME" : Done !')

    tgt_cur.close()
    tgt_conn.commit()
    

except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')
    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')



Job Start... 2025-02-17, 16:09:48

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

   -> data as of (YR_Start: 2023, YR_End: 0)

   -> src_df : 1096 rows, 46 columns

   -> Generate "dim_time.csv" successfully

   -> TRUNCATE : "DIM_TIME" : Done !

   -> INSERT : "DIM_TIME" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [6]:
src_df.tail()

,TM_KEY_DAY,DATE_ID,DATE_VALUE,DATE_DESC,DAY_DESC,DAY_SHORT,DAY_DESC_TH,DAY_SHORT_TH,DAY_NO,DAY_OF_WEEK,...,YEAR_SHORT_TH,PERIODFLAG,ETL_DATE,ETL_UPDATE,WEEK_MONTH,WEEK_MONTH_DESC,TRUE_WK_BEG_DATE,TRUE_WK_END_DATE,TRUE_WK_MTH_BEG_DATE,TRUE_WK_MTH_END_DATE
1091,20251227,46018,2025-12-27,27-12-2025,Saturday,Sat,เสาร์,ส.,27,6,...,68,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2025124,Dec 22-28,2025-12-22,2025-12-28,2025-12-22,2025-12-28
1092,20251228,46019,2025-12-28,28-12-2025,Sunday,Sun,อาทิตย์,อา.,28,7,...,68,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2025124,Dec 22-28,2025-12-22,2025-12-28,2025-12-22,2025-12-28
1093,20251229,46020,2025-12-29,29-12-2025,Monday,Mon,จันทร์,จ.,29,1,...,68,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2025125,Dec 29-31,2025-12-29,2026-01-04,2025-12-29,2025-12-31
1094,20251230,46021,2025-12-30,30-12-2025,Tuesday,Tue,อังคาร,อ.,30,2,...,68,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2025125,Dec 29-31,2025-12-29,2026-01-04,2025-12-29,2025-12-31
1095,20251231,46022,2025-12-31,31-12-2025,Wednesday,Wed,พุธ,พ.,31,3,...,68,EMQY,2024-01-30 16:04:35,2024-01-30 16:04:35,2025125,Dec 29-31,2025-12-29,2026-01-04,2025-12-29,2025-12-31


In [7]:
src_df.dtypes

TM_KEY_DAY                       int64
DATE_ID                          int64
DATE_VALUE              datetime64[ns]
DATE_DESC                       object
DAY_DESC                        object
DAY_SHORT                       object
DAY_DESC_TH                     object
DAY_SHORT_TH                    object
DAY_NO                           int64
DAY_OF_WEEK                      int64
DAYS_IN_MONTH                    int64
TM_KEY_WK                        int64
WEEK_DESC                       object
WEEK_YEAR                        int64
TRUE_TM_KEY_WK                   int64
TRUE_WEEK                        int64
TRUE_WEEK_DESC                  object
TM_KEY_MTH                       int64
MONTH_ID                         int64
MONTH_SHORT                     object
MONTH_DESC                      object
MONTH_SHORT_TH                  object
MONTH_DESC_TH                   object
MONTH_NO                         int64
MONTH_BEG_DATE          datetime64[ns]
MONTH_END_DATE          d